# Lab06 Multivariate Linear Regression

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [7]:
# 데이터
x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,90],
                            [96,98,100],
                            [73,66,70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371071
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

## nn.Module 적용

In [11]:
# 데이터
x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,90],
                            [96,98,100],
                            [73,66,70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
# W = torch.zeros((3,1), requires_grad=True)
# b = torch.zeros(1, requires_grad=True)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self,x):
        return self.linear(x)

model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    # hypothesis = x_train.matmul(W) + b
    hypothesis = model(x_train)
    
    # cost 계산
    # cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([-23.0405, -22.0658, -24.7049, -26.0440, -16.1569]) Cost: 37947.335938
Epoch    1/20 hypothesis: tensor([53.0348, 69.3706, 65.3892, 72.0661, 53.5859]) Cost: 11895.314453
Epoch    2/20 hypothesis: tensor([ 95.6268, 120.5623, 115.8296, 126.9944,  92.6322]) Cost: 3729.389160
Epoch    3/20 hypothesis: tensor([119.4726, 149.2226, 144.0694, 157.7467, 114.4926]) Cost: 1169.805542
Epoch    4/20 hypothesis: tensor([132.8231, 165.2683, 159.8799, 174.9639, 126.7314]) Cost: 367.511749
Epoch    5/20 hypothesis: tensor([140.2978, 174.2516, 168.7316, 184.6031, 133.5833]) Cost: 116.034943
Epoch    6/20 hypothesis: tensor([144.4827, 179.2810, 173.6874, 189.9998, 137.4194]) Cost: 37.210064
Epoch    7/20 hypothesis: tensor([146.8258, 182.0966, 176.4621, 193.0212, 139.5669]) Cost: 12.502449
Epoch    8/20 hypothesis: tensor([148.1377, 183.6729, 178.0155, 194.7128, 140.7691]) Cost: 4.757693
Epoch    9/20 hypothesis: tensor([148.8724, 184.5553, 178.8853, 195.6599, 141.4421]) 